In [37]:
import numpy as np
import cv2
import os
import imutils

In [38]:
def rescaleFrame(frame, scale=0.5):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dimensions = (width, height)
    return cv2.resize(frame, dimensions, interpolation=cv2.INTER_AREA)

def calculate_brightness(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate the mean brightness of the image
    brightness = np.mean(gray)

    return brightness

In [39]:
def find_parallel(lines):
    max_parallel_count = 0
    best_line_index = -1
    best_parallel_indices = []

    for i in range(len(lines)):
        parallel_indices = []

        for j in range(len(lines)):
            if i == j:
                continue

            # Extract line coordinates
            x1_i, y1_i, x2_i, y2_i = lines[i][0]
            x1_j, y1_j, x2_j, y2_j = lines[j][0]

            # Check if the lines are approximately parallel
            delta_x_i = x2_i - x1_i
            delta_y_i = y2_i - y1_i
            delta_x_j = x2_j - x1_j
            delta_y_j = y2_j - y1_j

            # Avoid division by zero
            if delta_x_i == 0 or delta_x_j == 0:
                continue

            slope_i = delta_y_i / delta_x_i
            slope_j = delta_y_j / delta_x_j
 
            if abs(slope_i - slope_j) < 0.05:
                parallel_indices.append(j)

        if len(parallel_indices) > max_parallel_count:
            max_parallel_count = len(parallel_indices)
            best_line_index = i
            best_parallel_indices = parallel_indices

    return best_line_index, best_parallel_indices

In [40]:
def detectBarcodes2(image, contour, showProcess = False):
    
    mask = np.zeros(image.shape, dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1)
    
    masked_image = cv2.bitwise_and(image, mask)
    cv2.imshow("Masked Image", masked_image)
    
    gray = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    
    blurred = cv2.medianBlur(gray, 1) 
    
    ddepth = cv2.CV_32F
    gradX = cv2.Sobel(blurred, ddepth=ddepth, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(blurred, ddepth=ddepth, dx=0, dy=1, ksize=-1)
    # subtract the y-gradient from the x-gradient
    
    edgeDetection = cv2.subtract(gradX, gradY)
    edgeDetection = cv2.convertScaleAbs(edgeDetection)
    
    if showProcess:
        cv2.imshow("Edge", edgeDetection)

    kernel = np.ones((3, 3), np.uint8)
    
    _, thresh = cv2.threshold(edgeDetection, 150, 255, cv2.THRESH_BINARY)
    
    if showProcess:
        cv2.imshow("Thresh", thresh)
    
    erode = cv2.erode(thresh, None, iterations= 1)
    dilate = cv2.dilate(erode, None, iterations= 1)
    
    # if showProcess:
    cv2.imshow("Erode", erode)
    cv2.imshow("Dilate", dilate)
    
    lines = cv2.HoughLinesP(image=erode,rho=1.25,theta=np.pi/180, threshold=100,lines=np.array([]), minLineLength=100, maxLineGap=10)

    blank = np.zeros(image.shape, dtype=np.uint8)
    
    blank_temp = blank.copy()
    if lines is None:
        print("No lines")
        return contour
    
    for i in range(len(lines)):
        cv2.line(blank_temp, (lines[i][0][0], lines[i][0][1]), (lines[i][0][2], lines[i][0][3]), (0, 0, 255), 3, cv2.LINE_AA)
    
    # if showProcess:
    cv2.imshow("All Lines", blank_temp)
    
    if lines is None or len(lines) < 10:
        return contour
    
    best_line_index, best_parallel_indices = find_parallel(lines)
    
    if best_line_index == -1:
        print("No parallel lines found.")
        return contour
    
    best_line = lines[best_line_index][0]
    print(f"Best line with index {best_line_index} has {len(best_parallel_indices)} parallel lines.")

    # Draw the best line
    cv2.line(blank, (best_line[0], best_line[1]), (best_line[2], best_line[3]), (0, 0, 255), 3, cv2.LINE_AA)

    # Draw all parallel lines
    for parallel_index in best_parallel_indices:
        parallel_line = lines[parallel_index][0]
        cv2.line(blank, (parallel_line[0], parallel_line[1]), (parallel_line[2], parallel_line[3]), (0, 0, 255), 3, cv2.LINE_AA)
    
    #Find max height and max width of the drawn parallel lines
    
    cv2.imshow("Parallel", blank)
    
    red_pixels_x = np.where(blank[:, :, 2] == 255)[1]

    # Find the minimum and maximum x-coordinates
    x_min = np.min(red_pixels_x)
    x_max = np.max(red_pixels_x)
    
    # Find the y-coordinates where red channel is 255
    red_pixels_y = np.where(blank[:, :, 2] == 255)[0]
    
    # Find the minimum and maximum y-coordinates
    y_min = np.min(red_pixels_y)
    y_max = np.max(red_pixels_y)
    
    # Print the results
    print("Min X:", x_min)
    print("Max X:", x_max)
    print("Min Y:", y_min)
    print("Max Y:", y_max)
    
    # Need to edit on original contour to keep the rotation!!!
    # cnt = np.array([(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)], dtype=np.int32)
    
    cnt = contour.copy()
        
    for point in cnt:
        x, y = point[0]
        
        if x < x_min:
            x = x_min
        elif x > x_max:
            x = x_max
         
        if y < y_min:
            y = y_min
        elif y > y_max:
            y = y_max
            
        point[0] = [x, y]
        
    if cv2.contourArea(cnt) < cv2.contourArea(contour)/2 or cv2.contourArea(cnt) > cv2.contourArea(contour)*0.9:
        return contour
    
    # Draw the contour on the image
    return cnt
    
    

In [41]:
def detectBarcodes(image, showProcess = False):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blurred = cv2.medianBlur(gray, 3)
    
    edgeDetection = cv2.Canny(blurred, 50, 150)
    edgeDetection = cv2.convertScaleAbs(edgeDetection)

    kernel = np.ones((7, 7), np.uint8)

    # Calculate the mean brightness of the image
    brightness = calculate_brightness(image)

    if brightness > 150:  # Adjust thresholds for bright images
        erode_dilate = edgeDetection
    else:
        erode_dilate = cv2.dilate(edgeDetection, kernel, iterations=1)
        erode_dilate = cv2.erode(erode_dilate, np.ones((7, 7), np.uint8), iterations=2)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
    closed = cv2.morphologyEx(erode_dilate, cv2.MORPH_CLOSE, kernel)

    cnts = cv2.findContours(
        closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    largestContourArea = cv2.contourArea(
        sorted(cnts, key=cv2.contourArea, reverse=True)[0])

    for cnt in cnts:
        # Filter contours based on area
        if cv2.contourArea(cnt) < (largestContourArea/2):
            continue
            
        # original = image.copy()
        cnt = detectBarcodes2(image, cnt, False)
        
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.intp(box)
        
        # Draw a bounding box around each detected region
        cv2.drawContours(image, [box], -1, (0, 255, 0), 3)
        
    cv2.imshow("Detected Barcode", image)
    
    if showProcess:
        cv2.imshow("1- Blurred", blurred)
        cv2.imshow("2- Canny", edgeDetection)
        cv2.imshow("3- Erode and Dilate", erode_dilate)
        cv2.imshow("4- Closed", closed)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [42]:
folderPath = 'Images'

# List all files in the folder
files = os.listdir(folderPath)

for file in files:
    # Check if the file is an image
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        # Construct the full path to the image
        image_path = os.path.join(folderPath, file)
        
        # Read the image
        image = cv2.imread(image_path)

        if image.shape[0] > 800 or image.shape[1] > 1500:
            image = rescaleFrame(image)

        detectBarcodes(image, False)

Best line with index 0 has 79 parallel lines.
Min X: 205
Max X: 396
Min Y: 144
Max Y: 299
Best line with index 40 has 18 parallel lines.
Min X: 358
Max X: 651
Min Y: 158
Max Y: 354
Best line with index 0 has 9 parallel lines.
Min X: 89
Max X: 226
Min Y: 65
Max Y: 94
Best line with index 0 has 79 parallel lines.
Min X: 31
Max X: 352
Min Y: 31
Max Y: 143
Best line with index 10 has 14 parallel lines.
Min X: 108
Max X: 380
Min Y: 30
Max Y: 239
Best line with index 23 has 64 parallel lines.
Min X: 1600
Max X: 1863
Min Y: 263
Max Y: 435
Best line with index 0 has 65 parallel lines.
Min X: 917
Max X: 1178
Min Y: 117
Max Y: 222
Best line with index 0 has 90 parallel lines.
Min X: 162
Max X: 355
Min Y: 246
Max Y: 400
Best line with index 0 has 26 parallel lines.
Min X: 107
Max X: 400
Min Y: 18
Max Y: 176
Best line with index 6 has 1 parallel lines.
Min X: 177
Max X: 207
Min Y: 196
Max Y: 323
Best line with index 0 has 118 parallel lines.
Min X: 192
Max X: 453
Min Y: 258
Max Y: 420
Best line wi